In [18]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Lambda, dot
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import urllib
import urllib.request
import collections
import os
import zipfile

import numpy as np
import tensorflow as tf


In [6]:
def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exi
similarity = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([processed_a, processed_b])sts(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [3]:
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    vocabulary = read_data(filename)
    print(vocabulary[:7])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
               
similarity = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([processed_a, processed_b])                                                 vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary


In [7]:
vocab_size = 10000
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)
print(data[:7])

window_size = 3
vector_dim = 300
epochs = 200000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

print(couples[:10], labels[:10])

Found and verified text8.zip
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse']
[5234, 3081, 12, 6, 195, 2, 3134]
[[6418, 1], [3387, 8577], [342, 411], [6991, 1809], [3995, 850], [9099, 193], [793, 2326], [3220, 80], [2074, 316], [301, 7]] [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]


In [14]:
import keras.layers
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

# setup a cosine similarity operation which will be output in a secondary model
#similarity = keras.layers.merge([target, context], mode='cos', dot_axes=0)



In [16]:
from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

similarity = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([target, context])

In [19]:

# now perform the dot product operation to get a similarity measure
dot_product = dot([target, context],  1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# create a secondary validation model to run our similarity checks during training
validation_model = Model(input=[input_target, input_context], output=similarity)

/home/raz/.virtualenvs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys
/home/raz/.virtualenvs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
  # This is added back by InteractiveShellApp.init_path()


In [20]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [ ]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    #if cnt % 10000 == 0:
    #    sim_cb.run_sim()

Iteration 0, loss=0.692456841468811
Iteration 100, loss=0.6906285285949707
Iteration 200, loss=0.6895407438278198
Iteration 300, loss=0.6849873661994934
Iteration 400, loss=0.6767352819442749
Iteration 500, loss=0.7059943675994873
Iteration 600, loss=0.705617368221283
Iteration 700, loss=0.7018423676490784
Iteration 800, loss=0.7056811451911926
Iteration 900, loss=0.6803345680236816
Iteration 1000, loss=0.6743648648262024
Iteration 1100, loss=0.7066075801849365
Iteration 1200, loss=0.7039297223091125
Iteration 1300, loss=0.6845319867134094
Iteration 1400, loss=0.7046218514442444
Iteration 1500, loss=0.6813658475875854
Iteration 1600, loss=0.699113667011261
Iteration 1700, loss=0.7045992016792297
Iteration 1800, loss=0.7146629095077515
Iteration 1900, loss=0.7158553004264832
Iteration 2000, loss=0.7202573418617249
Iteration 2100, loss=0.6681737899780273
Iteration 2200, loss=0.7170472145080566
Iteration 2300, loss=0.6744980216026306
Iteration 2400, loss=0.6828838586807251
Iteration 2500,

Iteration 20400, loss=0.7065203785896301
Iteration 20500, loss=0.697632908821106
Iteration 20600, loss=0.6887385249137878
Iteration 20700, loss=0.7039055824279785
Iteration 20800, loss=0.6936193108558655
Iteration 20900, loss=0.693603515625
Iteration 21000, loss=0.7045962810516357
Iteration 21100, loss=0.6988707780838013
Iteration 21200, loss=0.6942204833030701
Iteration 21300, loss=0.6788610219955444
Iteration 21400, loss=0.6917794942855835
Iteration 21500, loss=0.6866841316223145
Iteration 21600, loss=0.6979015469551086
Iteration 21700, loss=0.7146234512329102
Iteration 21800, loss=0.6784809827804565
Iteration 21900, loss=0.6771630048751831
Iteration 22000, loss=0.7042216062545776
Iteration 22100, loss=0.7008751630783081
Iteration 22200, loss=0.6940945386886597
Iteration 22300, loss=0.6869939565658569
Iteration 22400, loss=0.7564719915390015
Iteration 22500, loss=0.7111278176307678
Iteration 22600, loss=0.6654816269874573
Iteration 22700, loss=0.7065216302871704
Iteration 22800, loss

Iteration 40500, loss=0.6857470870018005
Iteration 40600, loss=0.6812928915023804
Iteration 40700, loss=0.7209535241127014
Iteration 40800, loss=0.6684120893478394
Iteration 40900, loss=0.7153651714324951
Iteration 41000, loss=0.6845314502716064
Iteration 41100, loss=0.687568724155426
Iteration 41200, loss=0.6532655954360962
Iteration 41300, loss=0.7005895972251892
Iteration 41400, loss=0.6805911660194397
Iteration 41500, loss=0.6614933013916016
Iteration 41600, loss=0.710245668888092
Iteration 41700, loss=0.7074423432350159
Iteration 41800, loss=0.6738035082817078
Iteration 41900, loss=0.7164839506149292
Iteration 42000, loss=0.6338579058647156
Iteration 42100, loss=0.6870269179344177
Iteration 42200, loss=0.6855315566062927
Iteration 42300, loss=0.6930157542228699
Iteration 42400, loss=0.6849755048751831
Iteration 42500, loss=0.6937453150749207
Iteration 42600, loss=0.7088779807090759
Iteration 42700, loss=0.690464973449707
Iteration 42800, loss=0.688122034072876
Iteration 42900, los

Iteration 60600, loss=0.7880762815475464
Iteration 60700, loss=0.6627045273780823
Iteration 60800, loss=0.6527108550071716
Iteration 60900, loss=0.6793900728225708
Iteration 61000, loss=0.6980006098747253
Iteration 61100, loss=0.6810923218727112
Iteration 61200, loss=0.7048943042755127
Iteration 61300, loss=0.6789871454238892
Iteration 61400, loss=0.7150876522064209
Iteration 61500, loss=0.676831841468811
Iteration 61600, loss=0.6595618724822998
Iteration 61700, loss=0.7490617632865906
Iteration 61800, loss=0.7459346652030945
Iteration 61900, loss=0.6441687941551208
Iteration 62000, loss=0.7679092288017273
Iteration 62100, loss=0.6362922787666321
Iteration 62200, loss=0.6371408104896545
Iteration 62300, loss=0.792228102684021
Iteration 62400, loss=0.6373658180236816
Iteration 62500, loss=0.705111026763916
Iteration 62600, loss=0.6988626718521118
Iteration 62700, loss=0.667564332485199
Iteration 62800, loss=0.6523952484130859
Iteration 62900, loss=0.6770322322845459
Iteration 63000, los

Iteration 80700, loss=0.6689028143882751
Iteration 80800, loss=0.6576455235481262
Iteration 80900, loss=0.7033814787864685
Iteration 81000, loss=0.6467910408973694
Iteration 81100, loss=0.6974924206733704
Iteration 81200, loss=0.6897394061088562
Iteration 81300, loss=0.6782220005989075
Iteration 81400, loss=0.6631688475608826
Iteration 81500, loss=0.5402296781539917
Iteration 81600, loss=0.6892668008804321
Iteration 81700, loss=0.6973071098327637
Iteration 81800, loss=0.7413008213043213
Iteration 81900, loss=0.7577636241912842
Iteration 82000, loss=0.48918992280960083
Iteration 82100, loss=0.722496509552002
Iteration 82200, loss=0.7125773429870605
Iteration 82300, loss=0.7059282660484314
Iteration 82400, loss=0.6817818284034729
Iteration 82500, loss=0.6791678071022034
Iteration 82600, loss=0.7291134595870972
Iteration 82700, loss=0.7118585109710693
Iteration 82800, loss=0.6699598431587219
Iteration 82900, loss=0.6554508209228516
Iteration 83000, loss=0.8786559700965881
Iteration 83100,